In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import statsmodels.api as sm
from scipy.stats import shapiro, kstest,normaltest
import pickle
import json
import os

### Step 1: Problem statement:

### Step 2: Data Gathering

In [2]:
df=pd.read_csv('CO2 Emissions_Canada.csv')
df

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244
...,...,...,...,...,...,...,...,...,...,...,...,...
7380,VOLVO,XC40 T5 AWD,SUV - SMALL,2.0,4,AS8,Z,10.7,7.7,9.4,30,219
7381,VOLVO,XC60 T5 AWD,SUV - SMALL,2.0,4,AS8,Z,11.2,8.3,9.9,29,232
7382,VOLVO,XC60 T6 AWD,SUV - SMALL,2.0,4,AS8,Z,11.7,8.6,10.3,27,240
7383,VOLVO,XC90 T5 AWD,SUV - STANDARD,2.0,4,AS8,Z,11.2,8.3,9.9,29,232


### Step 3: EDA & Feature Engineering

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Make                              7385 non-null   object 
 1   Model                             7385 non-null   object 
 2   Vehicle Class                     7385 non-null   object 
 3   Engine Size(L)                    7385 non-null   float64
 4   Cylinders                         7385 non-null   int64  
 5   Transmission                      7385 non-null   object 
 6   Fuel Type                         7385 non-null   object 
 7   Fuel Consumption City (L/100 km)  7385 non-null   float64
 8   Fuel Consumption Hwy (L/100 km)   7385 non-null   float64
 9   Fuel Consumption Comb (L/100 km)  7385 non-null   float64
 10  Fuel Consumption Comb (mpg)       7385 non-null   int64  
 11  CO2 Emissions(g/km)               7385 non-null   int64  
dtypes: flo

In [4]:
x=df.drop(['Make','Model','CO2 Emissions(g/km)'], axis=1)
x

,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg)
0,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33
1,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29
2,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48
3,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25
4,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27
...,...,...,...,...,...,...,...,...,...
7380,SUV - SMALL,2.0,4,AS8,Z,10.7,7.7,9.4,30
7381,SUV - SMALL,2.0,4,AS8,Z,11.2,8.3,9.9,29
7382,SUV - SMALL,2.0,4,AS8,Z,11.7,8.6,10.3,27
7383,SUV - STANDARD,2.0,4,AS8,Z,11.2,8.3,9.9,29


In [5]:
y=df[['CO2 Emissions(g/km)']]
y

,CO2 Emissions(g/km)
0,196
1,221
2,136
3,255
4,244
...,...
7380,219
7381,232
7382,240
7383,232


In [6]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Vehicle Class                     7385 non-null   object 
 1   Engine Size(L)                    7385 non-null   float64
 2   Cylinders                         7385 non-null   int64  
 3   Transmission                      7385 non-null   object 
 4   Fuel Type                         7385 non-null   object 
 5   Fuel Consumption City (L/100 km)  7385 non-null   float64
 6   Fuel Consumption Hwy (L/100 km)   7385 non-null   float64
 7   Fuel Consumption Comb (L/100 km)  7385 non-null   float64
 8   Fuel Consumption Comb (mpg)       7385 non-null   int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 519.4+ KB


In [7]:
x= pd.get_dummies(x, columns=['Vehicle Class'])

In [8]:
x = pd.get_dummies(x, columns=['Transmission'])

In [9]:
x

,Engine Size(L),Cylinders,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),Vehicle Class_COMPACT,Vehicle Class_FULL-SIZE,Vehicle Class_MID-SIZE,...,Transmission_AS8,Transmission_AS9,Transmission_AV,Transmission_AV10,Transmission_AV6,Transmission_AV7,Transmission_AV8,Transmission_M5,Transmission_M6,Transmission_M7
0,2.0,4,Z,9.9,6.7,8.5,33,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2.4,4,Z,11.2,7.7,9.6,29,True,False,False,...,False,False,False,False,False,False,False,False,True,False
2,1.5,4,Z,6.0,5.8,5.9,48,True,False,False,...,False,False,False,False,False,True,False,False,False,False
3,3.5,6,Z,12.7,9.1,11.1,25,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3.5,6,Z,12.1,8.7,10.6,27,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,2.0,4,Z,10.7,7.7,9.4,30,False,False,False,...,True,False,False,False,False,False,False,False,False,False
7381,2.0,4,Z,11.2,8.3,9.9,29,False,False,False,...,True,False,False,False,False,False,False,False,False,False
7382,2.0,4,Z,11.7,8.6,10.3,27,False,False,False,...,True,False,False,False,False,False,False,False,False,False
7383,2.0,4,Z,11.2,8.3,9.9,29,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [10]:
x.replace({True:1,False:0},inplace=True)
x

,Engine Size(L),Cylinders,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),Vehicle Class_COMPACT,Vehicle Class_FULL-SIZE,Vehicle Class_MID-SIZE,...,Transmission_AS8,Transmission_AS9,Transmission_AV,Transmission_AV10,Transmission_AV6,Transmission_AV7,Transmission_AV8,Transmission_M5,Transmission_M6,Transmission_M7
0,2.0,4,Z,9.9,6.7,8.5,33,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.4,4,Z,11.2,7.7,9.6,29,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1.5,4,Z,6.0,5.8,5.9,48,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,3.5,6,Z,12.7,9.1,11.1,25,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3.5,6,Z,12.1,8.7,10.6,27,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,2.0,4,Z,10.7,7.7,9.4,30,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7381,2.0,4,Z,11.2,8.3,9.9,29,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7382,2.0,4,Z,11.7,8.6,10.3,27,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7383,2.0,4,Z,11.2,8.3,9.9,29,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
x['Fuel Type'].unique()

array(['Z', 'D', 'X', 'E', 'N'], dtype=object)

In [12]:
x['Fuel Type'].replace({'Z':3, 'D':5, 'X':4, 'E':2, 'N':1},inplace=True)

In [13]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 50 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Engine Size(L)                          7385 non-null   float64
 1   Cylinders                               7385 non-null   int64  
 2   Fuel Type                               7385 non-null   int64  
 3   Fuel Consumption City (L/100 km)        7385 non-null   float64
 4   Fuel Consumption Hwy (L/100 km)         7385 non-null   float64
 5   Fuel Consumption Comb (L/100 km)        7385 non-null   float64
 6   Fuel Consumption Comb (mpg)             7385 non-null   int64  
 7   Vehicle Class_COMPACT                   7385 non-null   int64  
 8   Vehicle Class_FULL-SIZE                 7385 non-null   int64  
 9   Vehicle Class_MID-SIZE                  7385 non-null   int64  
 10  Vehicle Class_MINICOMPACT               7385 non-null   int6

##### There are no any null/missing values so missing values handling is not required

##### Here we are going to use Decision Tree Model, so Outlier Handling / Scaling is not required

### Step 4: Feature Selection

##### there are no any assumptions over data so we will be performing this step after model training and eveluation

### step 5 Model Training

In [15]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=1)
x_train

,Engine Size(L),Cylinders,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),Vehicle Class_COMPACT,Vehicle Class_FULL-SIZE,Vehicle Class_MID-SIZE,...,Transmission_AS8,Transmission_AS9,Transmission_AV,Transmission_AV10,Transmission_AV6,Transmission_AV7,Transmission_AV8,Transmission_M5,Transmission_M6,Transmission_M7
580,2.4,4,4,11.1,8.3,9.8,29,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3998,2.0,4,4,11.2,7.6,9.8,29,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2228,2.0,4,3,11.2,8.4,9.9,29,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2954,2.5,4,4,8.7,6.5,7.7,37,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,3.5,6,3,12.1,8.7,10.6,27,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,3.4,6,3,11.9,8.6,10.4,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5192,2.0,4,3,9.3,7.3,8.4,34,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3980,2.0,4,3,10.7,8.5,9.7,29,0,0,0,...,0,0,0,0,0,0,0,0,0,0
235,2.4,4,4,12.2,8.6,10.6,27,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
dt_reg=DecisionTreeRegressor()
dt_reg

DecisionTreeRegressor()

In [19]:
dt_reg.fit(x_train,y_train)

DecisionTreeRegressor()

In [21]:
y_pred=dt_reg.predict(x_test)
y_pred

array([202., 214., 174., ..., 177., 115., 194.])

In [22]:
y_pred_train = dt_reg.predict(x_train)
y_pred_train

array([225., 230., 232., ..., 226., 244., 246.])

### Step 6 Model Evaluation: model is being evaluated in this step

In [27]:
mse_test = mean_squared_error(y_test,y_pred)
mse_test

7.408273443834266

In [28]:
mae_test = mean_absolute_error(y_test,y_pred)
mae_test

1.5294978022804697

In [29]:
np.sqrt(mse_test)

2.721814366159872

In [30]:
mse_train = mean_squared_error(y_train,y_pred_train)
mse_train

1.2366056651728623

In [31]:
mae_train = mean_absolute_error(y_train,y_pred_train)
mae_train

0.42930610632878735

In [32]:
np.sqrt(mse_train)

1.1120277268003989

In [33]:
r2score_test=r2_score(y_test,y_pred)
r2score_test

0.9978242028977429

In [34]:
r2score_train=r2_score(y_train,y_pred_train)
r2score_train

0.9996392293300826

### Hyper parameter tuning

In [51]:
df_reg_h = DecisionTreeRegressor()
param_grid = {'criterion' : ['squared_error', 'absolute_error'],
             'max_depth' : np.arange(5,15),
             'min_samples_split' : np.arange(3,8),
             'min_samples_leaf' : np.arange(2,5)}
gscv = RandomizedSearchCV(df_reg_h,param_grid,cv=5,n_jobs=-1)

In [52]:
gscv.fit(x_train,y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error'],
                                        'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'min_samples_leaf': array([2, 3, 4]),
                                        'min_samples_split': array([3, 4, 5, 6, 7])})

In [53]:
gscv.best_estimator_

DecisionTreeRegressor(max_depth=9, min_samples_leaf=2, min_samples_split=3)

In [54]:
dt_reg_hyp = DecisionTreeRegressor(max_depth=9, min_samples_leaf=2, min_samples_split=3)
dt_reg_hyp.fit(x_train,y_train)

DecisionTreeRegressor(max_depth=9, min_samples_leaf=2, min_samples_split=3)

In [55]:
y_pred=dt_reg_hyp.predict(x_test)
y_pred_train = dt_reg_hyp.predict(x_train)
mse_test = mean_squared_error(y_test,y_pred)
print('mse_test', mse_test)
mae_test = mean_absolute_error(y_test,y_pred)
print('mae_test', mae_test)
print('RMSE Test', np.sqrt(mse_test))
r2score_test=r2_score(y_test,y_pred)
print('r2score_test',r2score_test)
mse_train = mean_squared_error(y_train,y_pred_train)
print('mse_train', mse_train)
mae_train = mean_absolute_error(y_train,y_pred_train)
print('mae_train', mae_train)
print('RMSE Train', np.sqrt(mse_train))
r2score_train=r2_score(y_train,y_pred_train)
print('r2score_train',r2score_train)

mse_test 7.369223276074442
mae_test 2.064009165325812
RMSE Test 2.71463133336268
r2score_test 0.9978356718645
mse_train 6.131035844703403
mae_train 1.8566362075012992
RMSE Train 2.476092858659263
r2score_train 0.9982113150770079


### Creating get data function, pickle and json data file

In [35]:
def get_input_row(make,model,Vehicle_Class, Engine_Size, Cylinders,Transmission, Fuel_Type
                  ,Fuel_Consumption_City1, Fuel_Consumption_Hwy1
                  ,Fuel_Consumption_Comb2, Fuel_Consumption_Comb3):
    df1=pd.DataFrame(np.zeros(shape=(50)))
    df1.index=x.columns
    df2=df1.T
    df2['Engine Size(L)']=Engine_Size
    df2['Cylinders']=Cylinders
    df2['Fuel Consumption City (L/100 km)']=Fuel_Consumption_City1
    df2['Fuel Consumption Hwy (L/100 km)']=Fuel_Consumption_Hwy1
    df2['Fuel Consumption Comb (L/100 km)']=Fuel_Consumption_Comb2
    df2['Fuel Consumption Comb (mpg)']=Fuel_Consumption_Comb3
    df2['Fuel Type']=Fuel_Type
    col_name='Vehicle Class_'+ Vehicle_Class    
    df2[col_name]= 1
    col_name1='Transmission_'+ Transmission
    df2[col_name1]=1
    df2['Fuel Type'].replace({'Z':3, 'D':5, 'X':4, 'E':2, 'N':1},inplace=True)
    return df2

In [56]:
input_df=get_input_row('Suraj', 'SUV', 'COMPACT', 2.0, 4, 'AS5', 'Z', 9.9, 6.7, 8.6, 33)
y_predicted = dt_reg_hyp.predict(input_df)
predicted_co2_emmission = y_predicted[0]

In [57]:
with open('DT_regression.pkl','wb') as f:
    pickle.dump(dt_reg_hyp,f)

In [39]:
dict1 = {'columns_x' : x.columns.to_list() }
with open('project_data.json','w') as f:
    json.dump(dict1,f)